In [2]:
import pickle
from tqdm.notebook import tqdm
from pororo import Pororo
from datasets import load_from_disk
from torch.utils.data import DataLoader, TensorDataset

model = Pororo(task="mt", lang="multi", model="transformer.large.multi.mtpg")

[Korean Sentence Splitter]: Initializing Kss...


In [3]:
import pickle as pickle
import os
import re
import itertools
from collections import Counter
from typing import Dict, List, Tuple

from transformers import AutoTokenizer
from konlpy.tag import Mecab

import pandas as pd
import random
import torch
from torch.utils.data import Dataset, Subset

In [47]:
check_eval = pd.read_csv('/opt/ml/dataset/train/train.csv')
print(type(check_eval['subject_entity'][0]))
print(check_eval['subject_entity'][0])

<class 'str'>
{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}


In [52]:
print(type(eval(check_eval['subject_entity'][0])['word']))
print(eval(check_eval['subject_entity'][0])['word'])
print(type(eval(check_eval['subject_entity'][0])))

<class 'str'>
비틀즈
<class 'dict'>


In [7]:
def cycling_translation_en(sentence):
    english = model(sentence, 'ko', 'en')
    korean = model(english, 'en', 'ko')
    return korean

In [7]:
def remove_special_char(sentence):
    """ 특수문자 및 독일어 제거, 수정"""
    sentence = re.sub(r'[À-ÿ]+', '', sentence)  # 독일어
    sentence = re.sub(r'[\u0600-\u06FF]+', '', sentence)  # 사우디어
    sentence = re.sub(r'[ß↔Ⓐب€☎☏±]+', '', sentence)
    sentence = re.sub('–', '─', sentence)
    sentence = re.sub('⟪', '《', sentence)
    sentence = re.sub('⟫', '》', sentence)
    sentence = re.sub('･', '・', sentence)
    return sentence


def add_space_char(sentence):
    def add_space(match):
        res_str = ', '.join(match.group().split(',')).rstrip()
        return res_str
    p = re.compile(r'([기-힣\w\-]+,)+[기-힣\w\-]+')
    sentence = p.sub(add_space, sentence)
    return sentence


def substitution_date(sentence):
    """
    기간 표시 '-' => '~'
    1223년 – => 1223년 ~ 
    """
    def sub_tibble(match):
        res_str = re.sub('[–\-]', '~', match.group())
        return res_str
    re_patterns = [
        r'(\d{2,4}년\s*)(\d{1,2}[월|일]\s*)(\d{1,2}[월|일])\s*[–\-]',
        r'(\d{2,4}년\s*)(\d{1,2}[월|일]\s*)\s*[–\-]',
        r'(\d{2,4}년\s*)\s*[–\-]',
        r'\((\d{4}[–\-]\d{2,4})\)'
    ]
    for re_pattern in re_patterns:
        p = re.compile(re_pattern)
        sentence = p.sub(sub_tibble, sentence)
    return sentence


def sentence_processing(data):
    new_sentence = []
    for row in tqdm(data.values):
        sentence, subject_entity, object_entity = row[1], eval(
            row[2]), eval(row[3])
        sub_start_idx, sub_end_idx, sub_type = subject_entity[
            'start_idx'], subject_entity['end_idx'], subject_entity['type']
        ob_start_idx, ob_end_idx, ob_type = object_entity[
            'start_idx'], object_entity['end_idx'], object_entity['type']

        if sub_start_idx < ob_start_idx:
            sentence = sentence[:sub_start_idx] + ' @ ◈ ' + sub_type + ' ◈ ' + sentence[sub_start_idx:sub_end_idx+1] + ' @ ' + \
                sentence[sub_end_idx+1:ob_start_idx] + ' # ↑ ' + ob_type + ' ↑ ' + \
                sentence[ob_start_idx:ob_end_idx+1] + \
                ' # ' + sentence[ob_end_idx+1:]
        else:
            sentence = sentence[:ob_start_idx] + ' # ↑ ' + ob_type + ' ↑ ' + sentence[ob_start_idx:ob_end_idx+1] + ' # ' + sentence[ob_end_idx +
                                                                                                                                    1:sub_start_idx] + ' @ ◈ ' + sub_type + ' ◈ ' + sentence[sub_start_idx:sub_end_idx+1] + ' @ ' + sentence[sub_end_idx+1:]

        sentence = re.sub('\s+', " ", sentence)
        new_sentence.append(sentence)

    print("Finish type entity processing!!!")

    return new_sentence


def mecab_processing(sentence):
    tokenizer = Mecab()

    tokens = tokenizer.morphs(sentence)
    mecab_sentence = " ".join(tokens)

    return mecab_sentence

In [8]:
def text_preprocessing(sentence):
    sent = remove_special_char(sentence)
    sent = substitution_date(sent)
    sent = add_space_char(sent)
    return sent


def preprocessing_dataset(dataset, entity_flag=0, preprocessing_flag=0, mecab_flag=0):
    """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    subject_entity = []
    object_entity = []

    # sentence에 entity 속성 추가
    for i, j in zip(dataset['subject_entity'], dataset['object_entity']):
        i = eval(i)['word']
        j = eval(j)['word']

        subject_entity.append(i)
        object_entity.append(j)

    if entity_flag:
        new_sentence = sentence_processing(dataset)
        dataset.sentence = new_sentence

    if preprocessing_flag and mecab_flag:
        out_dataset = pd.DataFrame({'id': dataset['id'],
                                    'sentence': [mecab_processing(text_preprocessing(sent)) for sent in dataset['sentence']],
                                    'subject_entity': [mecab_processing(text_preprocessing(entity)) for entity in subject_entity],
                                    'object_entity': [mecab_processing(text_preprocessing(entity)) for entity in object_entity],
                                    'label': dataset['label'], })
        print('Finish preprocessing and mecab !!!')

    elif preprocessing_flag and not mecab_flag:
        out_dataset = pd.DataFrame({'id': dataset['id'],
                                    'sentence': [text_preprocessing(sent) for sent in dataset['sentence']],
                                    'subject_entity': [text_preprocessing(entity) for entity in subject_entity],
                                    'object_entity': [text_preprocessing(entity) for entity in object_entity],
                                    'label': dataset['label'], })
        print('Finish data preprocessing!!!')

    elif mecab_flag and not preprocessing_flag:
        out_dataset = pd.DataFrame({'id': dataset['id'],
                                    'sentence': [mecab_processing(sent) for sent in dataset['sentence']],
                                    'subject_entity': [mecab_processing(entity) for entity in subject_entity],
                                    'object_entity': [mecab_processing(entity) for entity in object_entity],
                                    'label': dataset['label'], })
        print('Finish mecab preprocessing!!!')
    else:
        out_dataset = pd.DataFrame({'id': dataset['id'],
                                    'sentence': (dataset['sentence']),
                                    'subject_entity': (subject_entity),
                                    'object_entity': (object_entity),
                                    'label': dataset['label'], })
        print('None preprocessing')

    return out_dataset


def load_data(dataset_dir, entity_flag=0, preprocessing_flag=0, mecab_flag=0):
    """ csv 파일을 경로에 맡게 불러 옵니다. """
    pd_dataset = pd.read_csv(dataset_dir)
    if 'train' in dataset_dir:
        # 완전 중복 제거 42개
        pd_dataset = pd_dataset.drop_duplicates(
            ['sentence', 'subject_entity', 'object_entity', 'label'], keep='first')
        # 라벨링이 다른 데이터 제거
        pd_dataset = pd_dataset.drop(index=[6749, 8364, 22258, 277, 25094])
        pd_dataset = pd_dataset.reset_index(drop=True)
        print("Finish remove duplicated data")

    dataset = preprocessing_dataset(
        pd_dataset, entity_flag, preprocessing_flag, mecab_flag)
    return dataset


def tokenized_dataset(dataset, tokenizer, is_inference=False):
    """ tokenizer에 따라 sentence를 tokenizing 합니다."""
    concat_entity = []
    for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        concat_entity.append(temp)

    if is_inference:
        """ Roberta TTI_flag """
        if 'roberta' in tokenizer.name_or_path and not 'xlm' in tokenizer.name_or_path:
            tokenized_sentences = tokenizer(
                concat_entity,
                list(dataset['sentence']),
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=256,
                add_special_tokens=True,
                return_token_type_ids=False
            )
        else:
            tokenized_sentences = tokenizer(
                concat_entity,
                list(dataset['sentence']),
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=256,
                add_special_tokens=True
            )
    else:
        """ Roberta TTI_flag """
        if 'roberta' in tokenizer.name_or_path and not 'xlm' in tokenizer.name_or_path:
            tokenized_sentences = tokenizer(
                concat_entity,
                list(dataset['sentence']),
                truncation=True,
                max_length=256,
                add_special_tokens=True,
                return_token_type_ids=False
            )
        else:
            tokenized_sentences = tokenizer(
                concat_entity,
                list(dataset['sentence']),
                truncation=True,
                max_length=256,
                add_special_tokens=True
            )

    return tokenized_sentences

In [13]:
train_dataset = load_data("/opt/ml/dataset/train/train.csv",
                              entity_flag=0, preprocessing_flag=0, mecab_flag=0)

Finish remove duplicated data
None preprocessing


In [14]:
train_dataset.head()

,id,sentence,subject_entity,object_entity,label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,비틀즈,조지 해리슨,no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,민주평화당,대안신당,no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,광주FC,한국프로축구연맹,org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,아성다이소,박정부,org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,요미우리 자이언츠,1967,no_relation


In [22]:
check_sentence = train_dataset['sentence'][0]
check_sentence

'〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [23]:
cycling_translation_en(check_sentence)

"'뭔가'는 1969년 앨범 '아비 로드'에서 조지 해리슨과 비틀스가 쓴 곡이다."

In [25]:
model('〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.', 'ko', 'en')

'"""Something"" is a song written by George Harrison and Beatles on the 1969 album ""Abby Road."""'

In [19]:
cycling_translation_en('비틀즈') # 다르게 번역 되는 경우가 있음

'비틀즈'

In [21]:
model(model('비틀즈', 'ko', 'en'), 'en', 'ko')

'비틀즈'

In [44]:
def make_rtt(dataset):
    data_num = len(dataset)
    count = 0
    lbl_candidate_list = ['per:place_of_residence', 'per:other_family', 'per:place_of_birth',
                     'org:founded_by', 'per:product', 'per:siblings', 'org:political/religious_affiliation',
                     'per:religion', 'per:schools_attended', 'org:dissolved', 'org:number_of_employees/members',
                     'per:place_of_death', ]
    lbl_dict = dict.fromkeys(lbl_candidate_list, 0)
    for ind in range(data_num):
        _, sen, subj, obj, label = dataset.loc[ind]
        
        if label not in lbl_candidate_list: # 개수가 193개 초과인 라벨은 추가하지 않음
            continue
        rtt_sen = cycling_translation_en(sen)
        rtt_subj = cycling_translation_en(subj)
        rtt_obj = cycling_translation_en(obj)
        
        if rtt_subj not in rtt_sen or rtt_obj not in rtt_sen: # 번역된 subj, obj가 번역된 문장에 없으면 추가하지 않음
            continue
        
        lbl_dict[label] += 1
        count += 1
        print(f'[{ind} / {count} 개]\nORIGINAL: {sen}\nRTT: {rtt_sen}\nsubject: {rtt_subj}, object: {rtt_obj}, label: {label}')
        print(lbl_dict, '\n')

In [45]:
make_rtt(train_dataset)

[68 / 1 개]
ORIGINAL: 경성대학교 재학 시절 우완 투수 김기표와 입학 동기생이었는데 이 둘은 입단 당시 각각 '좌원삼', '우기표'로 불렸다.
RTT: 그는 경성대에 있을 때 대학 입학 당시 각각 '왼쪽 샘'과 '오른손잡이'로 불렸던 우익수 김기표와 동문이었다.
subject: 김기표, object: 경성대, label: per:schools_attended
{'per:place_of_residence': 0, 'per:other_family': 0, 'per:place_of_birth': 0, 'org:founded_by': 0, 'per:product': 0, 'per:siblings': 0, 'org:political/religious_affiliation': 0, 'per:religion': 0, 'per:schools_attended': 1, 'org:dissolved': 0, 'org:number_of_employees/members': 0, 'per:place_of_death': 0} 

[91 / 2 개]
ORIGINAL: 백한성(白漢成, 水原鶴人, 1899년 6월 15일 조선 충청도 공주 출생 ~ 1971년 10월 13일 대한민국 서울에서 별세.)은 대한민국의 정치가이며 법조인이다.
RTT: 백한성(1971년 10월 13일 한국 서울에서 사망한 15일 백한성, 수원학인, 충청도 공주)은 한국의 정치인이자 법조인이다.
subject: 백한성, object: 충청도 공주, label: per:place_of_birth
{'per:place_of_residence': 0, 'per:other_family': 0, 'per:place_of_birth': 1, 'org:founded_by': 0, 'per:product': 0, 'per:siblings': 0, 'org:political/religious_affiliation': 0, 'per:religion': 0, 'per:schools_attended': 1, 'org:dissolv

KeyboardInterrupt: 

In [53]:
check_dataset = train_dataset[:105]
check_dataset.tail()

,id,sentence,subject_entity,object_entity,label
100,100,이듬해 1997년에는 부상에서 회복된 채로 시즌을 맞이했으나 7월 1일 요미우리전에...,기요하라 가즈히로,1루수,per:title
101,101,"민 리(1977년 6월 27일 ~)는 베트남계 캐나다인 게임 개발자로, 1999년 ...",민 리,캐나다,per:origin
102,102,1959년 한국 전쟁으로 폐교된 대성여자중학교를 부활시키고 1960년에는 대성여자고...,대성여자중학교,1959년,org:dissolved
103,103,서울은퇴공무원상록자원봉사단은 2019년 4월 공무원연금공단(이사장 정남준) 서울지부...,공무원연금공단,정남준,org:top_members/employees
104,104,"1856년 주의 인가로 오번 시에 ""감리교파 대학교""(East Alabama Mal...",오번 대학교,1856년,org:founded


In [55]:
check_point_sen = check_dataset['sentence'][0]
check_point_sen += ' 이건 두 번째 문장이다.'
print('한 문장', check_dataset['sentence'][0])
print('문장 추가', check_point_sen)
print('한 문장 분리', check_dataset['sentence'][0].split('.'))
print('두 문장 분리', check_point_sen.split('.'))

한 문장 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
문장 추가 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다. 이건 두 번째 문장이다.
한 문장 분리 ['〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다', '']
두 문장 분리 ['〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다', ' 이건 두 번째 문장이다', '']


In [58]:
check_dataset = train_dataset[:105]

def make_rtt_csv(dataset):
    data_num = len(dataset)
    count = 0 # 총 몇 개의 데이터가 추가되는지 확인용
    lbl_candidate_list = ['per:place_of_residence', 'per:other_family', 'per:place_of_birth',
                     'org:founded_by', 'per:product', 'per:siblings', 'org:political/religious_affiliation',
                     'per:religion', 'per:schools_attended', 'org:dissolved', 'org:number_of_employees/members',
                     'per:place_of_death', ] # 개수가 193개 이하인 라벨들
    lbl_dict = dict.fromkeys(lbl_candidate_list, 0) # lbl_candidate_list를 키로 갖는 dictionary를 0으로 초기화
    
    ind_list = []
    new_sen_list = []
    new_subj_list = []
    new_obj_list = []
    label_list = []
    for ind in range(data_num):
        id, sen, subj, obj, label = dataset.loc[ind]
        
        if label not in lbl_candidate_list: # 개수가 193개 초과인 라벨은 추가하지 않음
            continue
        rtt_sen = cycling_translation_en(sen)
        rtt_subj = cycling_translation_en(subj)
        rtt_obj = cycling_translation_en(obj)
        
        if rtt_subj not in rtt_sen or rtt_obj not in rtt_sen: # 번역된 subj, obj가 번역된 문장에 없으면 추가하지 않음
            continue
        if len(sen.split('.')) < len(rtt_sen.split('.')): # 문장이 여러 개로 만들어졌으면 추가하지 않는다 (잘못 번역되었을 확률 높음)
            continue
        
        ind_list.append(count)
        new_sen_list.append(rtt_sen)
        new_subj_list.append(rtt_subj)
        new_obj_list.append(rtt_obj)
        label_list.append(label)
        
        lbl_dict[label] += 1
        count += 1
        
        print(f'[{ind} / {count} 개]\nORIGINAL: {sen}\nRTT: {rtt_sen}\nsubject: {rtt_subj}, object: {rtt_obj}, label: {label}')
        print(lbl_dict, '\n')
    new_data = {'sentence': new_sen_list,
               'subject_entity': new_subj_list,
               'object_entity': new_obj_list,
               'label': label_list}
    new_data_df = pd.DataFrame(new_data)
    new_data_df.to_csv('/opt/ml/dataset/train/check_rtt_data.csv')

In [59]:
make_rtt_csv(check_dataset)

[68 / 1 개]
ORIGINAL: 경성대학교 재학 시절 우완 투수 김기표와 입학 동기생이었는데 이 둘은 입단 당시 각각 '좌원삼', '우기표'로 불렸다.
RTT: 그는 경성대에 있을 때 대학 입학 당시 각각 '왼쪽 샘'과 '오른손잡이'로 불렸던 우익수 김기표와 동문이었다.
subject: 김기표, object: 경성대, label: per:schools_attended
{'per:place_of_residence': 0, 'per:other_family': 0, 'per:place_of_birth': 0, 'org:founded_by': 0, 'per:product': 0, 'per:siblings': 0, 'org:political/religious_affiliation': 0, 'per:religion': 0, 'per:schools_attended': 1, 'org:dissolved': 0, 'org:number_of_employees/members': 0, 'per:place_of_death': 0} 

[91 / 2 개]
ORIGINAL: 백한성(白漢成, 水原鶴人, 1899년 6월 15일 조선 충청도 공주 출생 ~ 1971년 10월 13일 대한민국 서울에서 별세.)은 대한민국의 정치가이며 법조인이다.
RTT: 백한성(1971년 10월 13일 한국 서울에서 사망한 15일 백한성, 수원학인, 충청도 공주)은 한국의 정치인이자 법조인이다.
subject: 백한성, object: 충청도 공주, label: per:place_of_birth
{'per:place_of_residence': 0, 'per:other_family': 0, 'per:place_of_birth': 1, 'org:founded_by': 0, 'per:product': 0, 'per:siblings': 0, 'org:political/religious_affiliation': 0, 'per:religion': 0, 'per:schools_attended': 1, 'org:dissolv

In [4]:
pd_dataset = pd.read_csv('/opt/ml/dataset/train/train.csv')[:105]
    
# 완전 중복 제거 42개
pd_dataset = pd_dataset.drop_duplicates(
    ['sentence', 'subject_entity', 'object_entity', 'label'], keep='first')
# 라벨링이 다른 데이터 제거
# pd_dataset = pd_dataset.drop(index=[6749, 8364, 22258, 277, 25094]) # test 용이라 지웠습니다
# pd_dataset = pd_dataset.reset_index(drop=True)
print("Finish remove duplicated data")

Finish remove duplicated data


In [9]:
def make_rtt_csv(dataset):
    data_num = len(dataset)
    count = 0 # 총 몇 개의 데이터가 추가되는지 확인용
    lbl_candidate_list = ['per:place_of_residence', 'per:other_family', 'per:place_of_birth',
                     'org:founded_by', 'per:product', 'per:siblings', 'org:political/religious_affiliation',
                     'per:religion', 'per:schools_attended', 'org:dissolved', 'org:number_of_employees/members',
                     'per:place_of_death', ] # 개수가 193개 이하인 라벨들
    lbl_dict = dict.fromkeys(lbl_candidate_list, 0) # lbl_candidate_list를 키로 갖는 dictionary를 0으로 초기화
    
    ind_list = []
    new_sen_list = []
    new_subj_list = []
    new_obj_list = []
    label_list = []
    source_list = []
    for ind in range(data_num):
        _, sen, subj_dict, obj_dict, label, source = dataset.loc[ind]
        
        if label not in lbl_candidate_list: # 개수가 193개 초과인 라벨은 추가하지 않음
            continue
        rtt_sen = cycling_translation_en(sen)
        rtt_subj = cycling_translation_en(eval(subj_dict)['word'])
        rtt_obj = cycling_translation_en(eval(obj_dict)['word'])
        
        if rtt_subj not in rtt_sen or rtt_obj not in rtt_sen: # 번역된 subj, obj가 번역된 문장에 없으면 추가하지 않음
            continue
        if len(sen.split('.')) < len(rtt_sen.split('.')): # 문장이 여러 개로 만들어졌으면 추가하지 않는다 (잘못 번역되었을 확률 높음)
            continue
            
        subj_start = rtt_sen.find(rtt_subj)
        subj_end = subj_start + len(rtt_subj) - 1
        rtt_subj = str(dict({'word':rtt_subj,
                            'start_idx':subj_start,
                            'end_idx':subj_end,
                            'type':eval(subj_dict)['type'],
                            }))
        obj_start = rtt_sen.find(rtt_obj)
        obj_end = obj_start + len(rtt_obj) - 1
        rtt_obj = str(dict({'word':rtt_obj,
                            'start_idx':obj_start,
                            'end_idx':obj_end,
                            'type':eval(subj_dict)['type'],
                           }))

        ind_list.append(count)
        new_sen_list.append(rtt_sen)
        new_subj_list.append(rtt_subj)
        new_obj_list.append(rtt_obj)
        label_list.append(label)
        source_list.append(source)
        lbl_dict[label] += 1
        count += 1
        
        print(f'[{ind} / {count} 개]\nORIGINAL: {sen}\nRTT: {rtt_sen}\nsubject: {rtt_subj}, object: {rtt_obj}, label: {label}')
        print(lbl_dict, '\n')
    new_data = {'sentence': new_sen_list,
               'subject_entity': new_subj_list,
               'object_entity': new_obj_list,
               'label': label_list,
               'source': source_list}
    new_data_df = pd.DataFrame(new_data)
    new_data_df.to_csv('/opt/ml/dataset/train/check_rtt_data.csv')

In [10]:
make_rtt_csv(pd_dataset)

[68 / 1 개]
ORIGINAL: 경성대학교 재학 시절 우완 투수 김기표와 입학 동기생이었는데 이 둘은 입단 당시 각각 '좌원삼', '우기표'로 불렸다.
RTT: 그는 경성대에 있을 때 대학 입학 당시 각각 '왼쪽 샘'과 '오른손잡이'로 불렸던 우익수 김기표와 동문이었다.
subject: {'word': '김기표', 'start_idx': 50, 'end_idx': 52, 'type': 'PER'}, object: {'word': '경성대', 'start_idx': 3, 'end_idx': 5, 'type': 'PER'}, label: per:schools_attended
{'per:place_of_residence': 0, 'per:other_family': 0, 'per:place_of_birth': 0, 'org:founded_by': 0, 'per:product': 0, 'per:siblings': 0, 'org:political/religious_affiliation': 0, 'per:religion': 0, 'per:schools_attended': 1, 'org:dissolved': 0, 'org:number_of_employees/members': 0, 'per:place_of_death': 0} 

[91 / 2 개]
ORIGINAL: 백한성(白漢成, 水原鶴人, 1899년 6월 15일 조선 충청도 공주 출생 ~ 1971년 10월 13일 대한민국 서울에서 별세.)은 대한민국의 정치가이며 법조인이다.
RTT: 백한성(1971년 10월 13일 한국 서울에서 사망한 15일 백한성, 수원학인, 충청도 공주)은 한국의 정치인이자 법조인이다.
subject: {'word': '백한성', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}, object: {'word': '충청도 공주', 'start_idx': 45, 'end_idx': 50, 'type': 'PER'}, label: per:place_of_birth
{'pe

In [11]:
check_rtt_data = pd.read_csv('/opt/ml/dataset/train/check_rtt_data.csv')
check_rtt_data

,Unnamed: 0,sentence,subject_entity,object_entity,label,source
0,0,그는 경성대에 있을 때 대학 입학 당시 각각 '왼쪽 샘'과 '오른손잡이'로 불렸던 ...,"{'word': '김기표', 'start_idx': 50, 'end_idx': 52...","{'word': '경성대', 'start_idx': 3, 'end_idx': 5, ...",per:schools_attended,wikipedia
1,1,"백한성(1971년 10월 13일 한국 서울에서 사망한 15일 백한성, 수원학인, 충...","{'word': '백한성', 'start_idx': 0, 'end_idx': 2, ...","{'word': '충청도 공주', 'start_idx': 45, 'end_idx':...",per:place_of_birth,wikipedia
2,2,"1959년 한국전쟁에서 문을 닫은 대성여중학교를 부활시켰고, 1960년 대성소녀고를...","{'word': '여중학교', 'start_idx': 21, 'end_idx': 2...","{'word': '1959년', 'start_idx': 0, 'end_idx': 4...",org:dissolved,wikipedia


In [13]:
 # Unnamed는 왜 rename으로 안바뀔까요ㅠ
check_rtt_data.columns = ['id', 'sentence', 'subject_entity', 'object_entity', 'label', 'source']
check_rtt_data

,id,sentence,subject_entity,object_entity,label,source
0,0,그는 경성대에 있을 때 대학 입학 당시 각각 '왼쪽 샘'과 '오른손잡이'로 불렸던 ...,"{'word': '김기표', 'start_idx': 50, 'end_idx': 52...","{'word': '경성대', 'start_idx': 3, 'end_idx': 5, ...",per:schools_attended,wikipedia
1,1,"백한성(1971년 10월 13일 한국 서울에서 사망한 15일 백한성, 수원학인, 충...","{'word': '백한성', 'start_idx': 0, 'end_idx': 2, ...","{'word': '충청도 공주', 'start_idx': 45, 'end_idx':...",per:place_of_birth,wikipedia
2,2,"1959년 한국전쟁에서 문을 닫은 대성여중학교를 부활시켰고, 1960년 대성소녀고를...","{'word': '여중학교', 'start_idx': 21, 'end_idx': 2...","{'word': '1959년', 'start_idx': 0, 'end_idx': 4...",org:dissolved,wikipedia
